WARNING: experimental FSI + .net standard

In [1]:
#r "netstandard"
#load "Paket.fsx"

In [2]:
Paket.Clear()
Paket.Version ["Microsoft.ML", "0.9.0"]

In [3]:
//Patch location for native dependencies
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML/runtimes/win-x64/native/")
    
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML.CpuMath/runtimes/win-x64/native/")

In [4]:
#load "Paket.Generated.Refs.fsx"

ML.NET Samples
--------------
https://github.com/dotnet/machinelearning-samples

In [5]:
Paket.GitHub ["dotnet/machinelearning-samples:073359c0802613430ddac45a10da4469342e5640 samples/fsharp/common_v0.9/ConsoleHelper.fs"]
Paket.GitHub ["dotnet/machinelearning-samples:073359c0802613430ddac45a10da4469342e5640 samples/fsharp/common_v0.9/ModelBuilder.fs"]
Paket.GitHub ["dotnet/machinelearning-samples:073359c0802613430ddac45a10da4469342e5640 samples/fsharp/common_v0.9/ModelScorer.fs"]
Paket.GitHub ["dotnet/machinelearning-samples:073359c0802613430ddac45a10da4469342e5640 samples/fsharp/common_v0.9/Pipeline.fs"]

In [16]:
#load """paket-files\dotnet\machinelearning-samples\samples\fsharp\common_v0.9\ConsoleHelper.fs"""
#load """paket-files\dotnet\machinelearning-samples\samples\fsharp\common_v0.9\ModelBuilder.fs"""
#load """paket-files\dotnet\machinelearning-samples\samples\fsharp\common_v0.9\ModelScorer.fs"""
#load """paket-files\dotnet\machinelearning-samples\samples\fsharp\common_v0.9\Pipeline.fs"""

Start training
-----

In [6]:
open Microsoft.ML
//open Microsoft.ML.Runtime.Api
open Microsoft.ML.Transforms
open Microsoft.ML.Trainers
open System
open Microsoft.ML.Data
//open SentimentAnalysis.DataStructures.Model

In [7]:
Util.Url "https://github.com/dotnet/machinelearning/blob/master/test/data/wikipedia-detox-250-line-data.tsv"

{ContentType = "text/html; charset=utf-8";
 Data =
  [|10uy; 10uy; 10uy; 10uy; 10uy; 10uy; 60uy; 33uy; 68uy; 79uy; 67uy; 84uy; 89uy;
    80uy; 69uy; 32uy; 104uy; 116uy; 109uy; 108uy; 62uy; 10uy; 60uy; 104uy; 116uy;
    109uy; 108uy; 32uy; 108uy; 97uy; 110uy; 103uy; 61uy; 34uy; 101uy; 110uy;
    34uy; 62uy; 10uy; 32uy; 32uy; 60uy; 104uy; 101uy; 97uy; 100uy; 62uy; 10uy;
    32uy; 32uy; 32uy; 32uy; 60uy; 109uy; 101uy; 116uy; 97uy; 32uy; 99uy; 104uy;
    97uy; 114uy; 115uy; 101uy; 116uy; 61uy; 34uy; 117uy; 116uy; 102uy; 45uy;
    56uy; 34uy; 62uy; 10uy; 32uy; 32uy; 60uy; 108uy; 105uy; 110uy; 107uy; 32uy;
    114uy; 101uy; 108uy; 61uy; 34uy; 100uy; 110uy; 115uy; 45uy; 112uy; 114uy;
    101uy; 102uy; 101uy; 116uy; 99uy; 104uy; ...|];}

In [8]:
let trainDataPath = @"wikipedia-detox-250-line-data.tsv"
let testDataPath = @"wikipedia-detox-250-line-test.tsv"

In [9]:
let mlContext = MLContext(seed = Nullable 1)

In [10]:
let textLoader =
    mlContext.Data.CreateTextReader (
        columns = 
            [|
                TextLoader.Column("Label", Nullable DataKind.Bool, 0)
                TextLoader.Column("Text", Nullable DataKind.Text, 1)
            |],
        hasHeader = true,
        separatorChar = '\t'
    )

let trainingDataView = textLoader.Read trainDataPath
let testDataView = textLoader.Read testDataPath

In [25]:
// STEP 2: Common data process configuration with pipeline data transformations          
let dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Text", "Features")

In [24]:
// (OPTIONAL) Peek data (such as 2 records) in training DataView after applying the ProcessPipeline's transformations into "Features" 
//Common.ConsoleHelper.peekDataViewInConsole<SentimentIssue> mlContext trainingDataView dataProcessPipeline 2 |> ignore
//Common.ConsoleHelper.peekVectorColumnDataInConsole mlContext "Features" trainingDataView dataProcessPipeline 1 |> ignore


The type 'SentimentIssue' is not defined.

In [17]:
// STEP 3: Set the training algorithm, then create and config the modelBuilder                            
let trainer = mlContext.BinaryClassification.Trainers.FastTree(labelColumn = "Label", featureColumn = "Features")
let modelBuilder =
    Common.ModelBuilder.create mlContext dataProcessPipeline
    |> Common.ModelBuilder.addTrainer trainer

In [18]:
 // STEP 4: Train the model fitting to the DataSet
printfn "=============== Training the model ==============="
let trainedModel = 
    modelBuilder
    |> Common.ModelBuilder.train trainingDataView

=============== Training the model ===============


In [21]:
// STEP 5: Evaluate the model and show accuracy stats
printfn "===== Evaluating Model's accuracy with Test data ====="
let predictions = trainedModel.Transform testDataView
let metrics = mlContext.BinaryClassification.Evaluate(predictions, "Label", "Score")


===== Evaluating Model's accuracy with Test data =====


In [22]:
Common.ConsoleHelper.printBinaryClassificationMetrics (trainer.ToString()) metrics

************************************************************
*       Metrics for Microsoft.ML.Trainers.FastTree.FastTreeBinaryClassificationTrainer binary classification model      
*-----------------------------------------------------------
*       Accuracy: 88.89%
*       Auc:      97.53%
*       Auprc:    97.88%
*       F1Score:  90.00%
*       LogLogg:  0.55%
*       LogLossreduction:  45.38%
*       PositivePrecision:      0.82
*       PositiveRecall:      1.00
*       NegativePrecision:      1.00
*       NegativeRecall:      0.78
************************************************************


In [23]:
//(trainedModel, modelBuilder)
//|> Common.ModelBuilder.saveModelAsFile modelPath

The value or constructor 'modelPath' is not defined. Maybe you want one of the following:
   Model